## **Exploring Spotify PlayLists for Track Identification**

In this note book we deploy Soptify API (spotpy) to obtain: 1) all the playlists from a given user id and 2) specific playlist shared by the user

In [158]:
#import spotipy with a developers id and passcode

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="07ca03cba8534f969d080936d5833062"
 
secret = "fbfcbbd68f884d5e887d7b502f97a036"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [159]:
#usual imports

import pandas as pd


In [160]:
#import survey results
survey = pd.read_csv('survey_final.csv', header=0)

In [161]:
survey=survey.drop([0])
survey=survey.drop([1])
survey.columns


Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q8', 'Q1_1', 'Q1_2', 'Q1_3',
       'Q3_1', 'Q3_2', 'Q3_3', 'Q1_2 - Topics', 'Q3_1 - Topics'],
      dtype='object')

In [191]:
survey.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,UserLanguage,Q8,Q1_1,Q1_2,Q1_3,Q3_1,Q3_2,Q3_3,Q1_2 - Topics,Q3_1 - Topics
2,11/1/18 18:11,11/1/18 18:34,IP Address,71.232.23.32,100,1376,TRUE,11/1/18 18:34,R_3e8NCLhxBzvrO8r,NaN,...,EN,Agree,Erin Williams,erinanthonywilliams@gmail.com,neqe9xtke1qxibqhma4se1vwx,https://open.spotify.com/user/neqe9xtke1qxibqh...,https://open.spotify.com/user/neqe9xtke1qxibqh...,https://open.spotify.com/user/neqe9xtke1qxibqh...,NaN,Unknown
3,11/1/18 19:07,11/1/18 19:14,IP Address,68.34.133.37,100,382,TRUE,11/1/18 19:14,R_R3SlkTHchuvZsL7,NaN,...,EN,Agree,Nadege Benoit,Nbenoit36@gmail.com,Nbenoit36,https://open.spotify.com/user/spotify/playlist...,https://open.spotify.com/user/1221022833/playl...,https://open.spotify.com/user/alena_gorbunova/...,NaN,Unknown
4,11/1/18 19:23,11/1/18 19:26,IP Address,74.196.230.56,100,176,TRUE,11/1/18 19:26,R_1QatFWOKknPGxLj,NaN,...,EN,Agree,Bonnie Huggins,Bonita.a.huggins@gmail.com,Bonnie19belle,https://open.spotify.com/user/bonnie19belle/pl...,https://open.spotify.com/user/bonnie19belle/pl...,https://open.spotify.com/user/digster.fm/playl...,NaN,Unknown
5,11/1/18 19:45,11/1/18 19:51,IP Address,24.21.27.16,100,338,TRUE,11/1/18 19:51,R_1KwgND3pykpGm3D,NaN,...,EN,Agree,Sandra Nippert,Smidjy@aol.com,Sandy Nippert,https://open.spotify.com/user/nickfearless/pla...,https://open.spotify.com/user/giesen.tobi/play...,https://open.spotify.com/user/maranathamusicof...,NaN,Unknown
6,11/1/18 19:48,11/1/18 19:52,IP Address,144.163.26.57,100,262,TRUE,11/1/18 19:52,R_2ATsllX5d8iTw4e,NaN,...,EN,Agree,Sara Bell,Sarakbell2018@gmail.com,skayl6,https://open.spotify.com/user/skayl6/playlist/...,https://open.spotify.com/user/skayl6/playlist/...,https://open.spotify.com/user/skayl6/playlist/...,NaN,Unknown


In [162]:
#get tracks from shared playlists
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    return results

In [163]:
sp.user_playlist(username,playlist_id)['name']

'Folk Pop'

In [164]:
all_users=[]
user_all_playlists=[]
cols=['Q3_1','Q3_2','Q3_3']
playlist_ids=[]
playlist_names=[]
i=2
while i <(len(survey)+2):
    username=str(survey.loc[i,['Q1_3']].values[0])
    k=0
    while k<3:
        playlist_link=str(survey.loc[i,[cols[k]]].values[0])
        playlist_id=playlist_link.split('/')[-1]
        playlist_name=sp.user_playlist(username,playlist_id)['name']
        playlist_names.append(playlist_name)
        playlist_ids.append(playlist_id)
        playlist=get_playlist_tracks(username,playlist_id) 
        track_uri=[]
        tracks=playlist['tracks']['items']
        m=0
        while m <len(tracks):
            uri=tracks[m]['track']['uri']
            track_uri.append(uri)
            m=m+1
        user_all_playlists.append(track_uri)
        k=k+1
    all_users.append(user_all_playlists)
    i=i+1
print("number of participants:",len(all_users))

number of participants: 43


In [165]:
all_users_tracks=all_users[0]
all_users_tracks=np.array(all_users_tracks).reshape(-1,1)

In [171]:
new_format=pd.DataFrame(columns=['UserEmail', 'PlaylistName','PlaylistID','Position', 'TrackURI'])

i=0
j=0
n=0
m=0
while i < 43:
    k=0
    while k<3:
        m=0
        while m < (len(all_users_tracks[j+k][0])):
            new_format.loc[n,['TrackURI']] = all_users_tracks[j+k][0][m]
            new_format.loc[n,['Position']]=m+1
            new_format.loc[n,['PlaylistID']]=playlist_ids[j+k]
            new_format.loc[n,['UserEmail']]=survey.Q1_2[i+2]
            new_format.loc[n,['PlaylistName']]=playlist_names[j+k]
            m=m+1
            n=n+1
    
        k=k+1
    j=j+3
    i=i+1
    
     

In [172]:
print(n)

7566


In [192]:
new_format.head()

,UserEmail,PlaylistName,PlaylistID,Position,TrackURI
0,erinanthonywilliams@gmail.com,Chill Out,5IPYYLhZB3vpB3nWJqpU0y?si=WJb1Ri1aRVOtw9W0Vmue3A,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v
1,erinanthonywilliams@gmail.com,Chill Out,5IPYYLhZB3vpB3nWJqpU0y?si=WJb1Ri1aRVOtw9W0Vmue3A,2,spotify:track:1Slwb6dOYkBlWal1PGtnNg
2,erinanthonywilliams@gmail.com,Chill Out,5IPYYLhZB3vpB3nWJqpU0y?si=WJb1Ri1aRVOtw9W0Vmue3A,3,spotify:track:6fxVffaTuwjgEk5h9QyRjy
3,erinanthonywilliams@gmail.com,Chill Out,5IPYYLhZB3vpB3nWJqpU0y?si=WJb1Ri1aRVOtw9W0Vmue3A,4,spotify:track:7soJgKhQTO8hLP2JPRkL5O
4,erinanthonywilliams@gmail.com,Chill Out,5IPYYLhZB3vpB3nWJqpU0y?si=WJb1Ri1aRVOtw9W0Vmue3A,5,spotify:track:1LJo5Y0TXHYvLwu22T6ibI


In [187]:
new_format.to_csv('survey_data_final.csv')

In [174]:
#just checking...
anna=new_format.loc[new_format['UserEmail'] == 'Anna.Davydova@gmail.com']

In [193]:
anna.head()

,UserEmail,PlaylistName,PlaylistID,Position,TrackURI
7467,Anna.Davydova@gmail.com,EDC,1nUXx2NFpwqpYrTvCp7ZvZ?si=CTQK190ETVqSEvcd_-jLDA,1,spotify:track:6wb61u8ayuanWSHZzle5od
7468,Anna.Davydova@gmail.com,EDC,1nUXx2NFpwqpYrTvCp7ZvZ?si=CTQK190ETVqSEvcd_-jLDA,2,spotify:track:3G0awBmyUu0eLh30XTthBk
7469,Anna.Davydova@gmail.com,EDC,1nUXx2NFpwqpYrTvCp7ZvZ?si=CTQK190ETVqSEvcd_-jLDA,3,spotify:track:76OSY26fhv9aJ0WZTezF34
7470,Anna.Davydova@gmail.com,EDC,1nUXx2NFpwqpYrTvCp7ZvZ?si=CTQK190ETVqSEvcd_-jLDA,4,spotify:track:5MGeW5mHnqeJiM0NPBbkEZ
7471,Anna.Davydova@gmail.com,EDC,1nUXx2NFpwqpYrTvCp7ZvZ?si=CTQK190ETVqSEvcd_-jLDA,5,spotify:track:0BOtWmOkV0BfytMJpT7I09


In [194]:
all_users_tracks=pd.DataFrame(all_users_tracks)
all_users_tracks.columns=["Playlist Tracks"]
all_users_tracks.head()


ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [178]:
index_user=[]
vals=list(survey['Q1_1'])
print(vals)
len(vals)

['Erin Williams', 'Nadege Benoit', 'Bonnie Huggins', 'Sandra Nippert', 'Sara Bell', 'Meredith Decker-Hamm', 'Jennifer Pierce', 'Bridget', 'Emily Schultz', 'Brittany Ott', 'Heather Lindsey', 'Eileen McGivney', 'Laura Yuengert', 'Sheng Wang', 'Kristen Stiles', 'Amanda Fredrickson', 'Whitney Polk', 'Delaney Marbach', 'JesÃºs M HernÃ¡ndez Ortizrtiz', 'Michael S. Emanuel', 'Rebecca Donnelly', 'Fiona Kenney', 'Sarah Shewell', 'Becky Jarvis', 'Antoinette Smith', 'Betsy chapman', 'Lydia Parra', 'Ryan Chappell', 'Jovonna', 'SUNNY LAN', 'Keita Broadwater', 'Stella anderson ', 'Nathan Paul', 'Carrie Kistler', 'Jennifer Rosauer', 'Chris Chen', 'Denise Bass', 'Jenessa White', 'Amanda Ciocci', 'Donald Jones', 'Garrett Beer', 'Anna Davydova', 'Christie Lee Gibson']


43

In [179]:
i=0
while i <43:
    n=0
    while n<3:
        m=vals[i]
        index_user.append(m)
        n=n+1
    i=i+1

In [180]:
index_user=pd.DataFrame(index_user)

In [181]:
all_users_tracks=pd.merge(all_users_tracks,index_user, right_index=True, left_index=True)

In [182]:
all_users_tracks.columns=["Playlist Tracks", "User"]

In [195]:
#file with all the track uri's per playlist, per user
all_users_tracks.head()

,Playlist Tracks,User
0,"[spotify:track:0tgVpDi06FyKpA1z0VMD4v, spotify...",Erin Williams
1,"[spotify:track:09IStsImFySgyp0pIQdqAc, spotify...",Erin Williams
2,"[spotify:track:0B4E0IrxBG77W80ZJQTy50, spotify...",Erin Williams
3,"[spotify:track:0bAkKNCQfWkexHFn7fIKns, spotify...",Nadege Benoit
4,"[spotify:track:2YEr07055St40kz0aUfm4f, spotify...",Nadege Benoit


In [186]:
all_users_tracks.loc[128,['Playlist Tracks']][0]
#spotify:track:1i1fxkWeaMmKEB4T7zqbzK

['spotify:track:3oLKEO5NCdf2Zi3mBzKaAs',
 'spotify:track:0uHYplBhwLYey7f9qAmnSM',
 'spotify:track:6UkMcAA19lTdjs22jtB7o2',
 'spotify:track:7shVwhUdVbHpykOfbzvDc1',
 'spotify:track:1pjATX7sbd6Y4jMVqIvzHk',
 'spotify:track:3LcYYV9ozePfgYYmXv0P3r',
 'spotify:track:2T5Ch09nefwckOu5NQvjIk',
 'spotify:track:69l8nFPu0LWmnoJX7YSVYc',
 'spotify:track:2xLU9wcdlFtuCIUDn05d2p',
 'spotify:track:04x9RfuAP5Pyce6fYzK9Xp',
 'spotify:track:2VESf4DiwSf8nne5kMVTpa',
 'spotify:track:2sbF2F1fIKylqW79XFLbH8',
 'spotify:track:1pmYAWY7Kwaj84U3xZxLML',
 'spotify:track:7qFg0joTjCiYOvzrt9nNtg',
 'spotify:track:19slC7k8bsPOAKDjHYLU2W',
 'spotify:track:6Xz7FeyE8HTP90HecgHV57',
 'spotify:track:1ppuHX1oVMku5LTL0swNZP',
 'spotify:track:2obblQ6tcePeOEVJV6nEGD',
 'spotify:track:7qnxmC6qdYkjcoCJCeD8Jx']

In [40]:
# define a user friendly way to display tracks

def show_tracks(results):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %32.32s %s" % (i, track['artists'][0]['name'], track['name']))

In [41]:
#get all the tracks from all the playlists that the user has

username=username
playlists = sp.user_playlists(username)
for playlist in playlists['items']:
        if playlist['owner']['id'] == username:
            print()
            print(playlist['name'])   
            print('  total tracks', playlist['tracks']['total'])
            results = sp.user_playlist(username, playlist['id'], fields="tracks,next")
            
            tracks = results['tracks']
            show_tracks(tracks)
            while tracks['next']:
                tracks = sp.next(tracks)
                show_tracks(tracks)



French concert
  total tracks 19
   0                       Édith Piaf Milord - Remasterisé en 2015
   1                    Carole Farley Early In The Morning
   2                      Léo Delibes Les Filles de Cadix
   3                  Francis Poulenc Nos souvenirs chantent, FP 182
   4                       Édith Piaf L'accordéoniste - Live;Remasterisé en 2015
   5                       Édith Piaf Les amants d'un jour - Remasterisé en 2015
   6                   Jules Massenet Manon - Acte II : "Adieu, notre petite table" Manon
   7                   Jules Massenet Massenet: Manon, Act 3 Tableau 1 Scene 4: "Obéissons, quand leur voix appelle … Profitons bien de la jeunesse" (Manon, Brétigny, Chorus) - Gavotte
   8                  Ambroise Thomas Mignon / Act 1: "Connais-tu le pays"
   9                   Charles Gounod Depuis hier je cherche en vain mon maître (Récit). Que fais-tu, blanche tourterelle (Chanson)
   10                Jacques Offenbach Les Contes d'Hoffmann / Act 2:

   0 Jessie Norman, Boulez, Pierre, d Sheherazade
   1 Jessie Norman, Boulez, Pierre, d Trois poemes de Stephane Mallarme
   2 Jessie Norman, Boulez, Pierre, d Chansons Medecasses
   3 Jessie Norman, Boulez, Pierre, d Don Quichotte a Dulcinee
   4 Jessie Norman, Boulez, Pierre, d Cinq Melodies populaires Grecques

Auditions 2013
  total tracks 2
   0                                  Gavotte_mixdown
   1                                  Silver_mixdown

Auditions 2018
  total tracks 20
   0              World Premiere Cast Overture
   1              World Premiere Cast Right Before Your Eyes
   2              World Premiere Cast On Your Way Home
   3              World Premiere Cast Property of Spiker and Sponge
   4              World Premiere Cast Shake It Up
   5              World Premiere Cast There’s Money On That Tree
   6              World Premiere Cast Middle Of A Moment
   7              World Premiere Cast Floatin’ Along
   8              World Premiere Cast Entr'acte
   9   

   0                                  05 Time In A Bottle

Labor - Soothing Songs
  total tracks 10
   0                    Joni Mitchell All I Want (LP Version)
   1                    Joni Mitchell My Old Man (LP Version)
   2                    Joni Mitchell Little Green (LP Version)
   3                    Joni Mitchell Carey (LP Version)
   4                    Joni Mitchell Blue (LP Version)
   5                    Joni Mitchell California (LP Version)
   6                    Joni Mitchell This Flight Tonight (LP Version)
   7                    Joni Mitchell River (LP Version)
   8                    Joni Mitchell A Case Of You (LP Version)
   9                    Joni Mitchell The Last Time I Saw Richard (LP Version)

Lessons
  total tracks 1
   0                                  KathrynW Nun beut 052116

Magic Mirror
  total tracks 8
   0                                  Aria of the Mother-Queen-edits
   1                                  transition scene with the chorus
   2 

   0                    Golijov_Spano Preludio de Agua y Caballo
   1                    Golijov_Spano I - Balada
   2                    Golijov_Spano I - Mariana, tu ojos
   3                    Golijov_Spano I - Bar Albor de Madrid
   4                    Golijov_Spano I - Desde mi ventana (Aria a la estatua di Mariana)
   5                    Golijov_Spano I - Muerte a Caballo
   6                    Golijov_Spano II - Balada
   7                    Golijov_Spano II - Quiero arrancarme los ojos
   8                    Golijov_Spano II - A la Habana
   9                    Golijov_Spano II - Quiero cantar entre las explosiones
   10                    Golijov_Spano II - Arresto
   11                    Golijov_Spano II - La Fuente de las Lágrimas
   12                    Golijov_Spano II - Confesión
   13                    Golijov_Spano II - Interludio de Balazos y Lamento por la Muerte de Federico
   14                    Golijov_Spano III - Balada
   15                    Golijov

   0 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Se Vuoi Ch'Io Mora
   1 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Amor S'Apprende
   2 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Perduta Ho La Speranza
   3 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Ah, Mai Non Cessate
   4 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Date Abbiento Al Mio Dolore
   5 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Sento Nel Core
   6 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Spirate Pur, Spirate
   7 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - Come L'Allodoletta
   8 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 1 - E Filli M'Ha Detto
   9 Roberto Guarino, Dennis Helmrich Donaudy: Amor Mi Fa Cantare, Book 2 - Freschi Luoghi, Prati Aulenti
   10 Roberto Guarino, Dennis 

   0 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Mesicku Na Nebi Hlubokem
   1 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Staleta Moudrost Tva
   2 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Cury Mury Fuk
   3 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Zde Mihla Se
   4 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Ustante V Lovu, Na Hrad Wrafte Se
   5 Richard Novac, Daniela Sounova,  Dvořák: Rusalka, Op. 114, B 203 - Vim, Ze Jsi Kouzlo, Ktere Mine
   6                 Jindrák, Marková Jarku, jarku (60)
   7                 Ochman, Drobková Jiz tyden dlis mi po boku (64)
   8       Benacková, Drobkova, Novak Festival music (68)
   9                            Novák Cely svet neda ti, neda (68)
   10 Prague Philharmonic Chorus, Nová Kvetiny bile po ceste (70)
   11                 Novák, Benacková Rusalka, znas mne, znas (70)
   12                        Benacková O marno

In [42]:
playlists

{'href': 'https://api.spotify.com/v1/users/7zbhtotv4oi6jlwlda9iy8fy0/playlists?offset=0&limit=50',
 'items': [{'collaborative': False,
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/3qIGgvWm2dod1tzqzGJJ91'},
   'href': 'https://api.spotify.com/v1/playlists/3qIGgvWm2dod1tzqzGJJ91',
   'id': '3qIGgvWm2dod1tzqzGJJ91',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/123de18d5de71cdd6f5c17b7c36c49d29190e719772030b1b7c7a34bd946e89b44b9a9e78cc3101190439faa1cdc9d7a817a8fbcf563abc9fc7f437bea68f1b8bd217b06d8344187651fb8eeda098b47',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/123de18d5de71cdd6f5c17b7c36c49d29190e719772030b1b7c7a34bd946e89b44b9a9e78cc3101190439faa1cdc9d7a817a8fbcf563abc9fc7f437bea68f1b8bd217b06d8344187651fb8eeda098b47',
     'width': 300},
    {'height': 60,
     'url': 'https://mosaic.scdn.co/60/123de18d5de71cdd6f5c17b7c36c49d29190e719772030b1b7c7a34bd946e89b44b9a9e78cc3101190439faa1cdc9d7a817a8fbcf563a

In [79]:
tracks1=tracks['items']
tracks1

[{'added_at': '2018-10-31T15:02:19Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/wu8kq8i7s861vw8f5o071slsk'},
   'href': 'https://api.spotify.com/v1/users/wu8kq8i7s861vw8f5o071slsk',
   'id': 'wu8kq8i7s861vw8f5o071slsk',
   'type': 'user',
   'uri': 'spotify:user:wu8kq8i7s861vw8f5o071slsk'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4NJhFmfw43RLBLjQvxDuRS'},
      'href': 'https://api.spotify.com/v1/artists/4NJhFmfw43RLBLjQvxDuRS',
      'id': '4NJhFmfw43RLBLjQvxDuRS',
      'name': 'Wolfgang Amadeus Mozart',
      'type': 'artist',
      'uri': 'spotify:artist:4NJhFmfw43RLBLjQvxDuRS'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/60adCptqwRkANTtVja0bvf'},
      'href': 'https://api.spotify.com/v1/artists/60adCptqwRkANTtVja0bvf',
      'id': '60adCptqwRkANTtVja0bvf',
      'name': 'Academy of Ancient Music

In [174]:
track1_uri=[]
list_of_all_uris=[]
for item in tracks['items']:
    i=0
    while i <len(tracks1):
        uri=tracks1[i]['track']['uri']
        track1_uri.append(uri)
        i=i+1
    list_of_all_uris.append(track1_uri)

In [175]:
list_of_all_uris

[['spotify:track:0B4E0IrxBG77W80ZJQTy50',
  'spotify:track:1O5u9GJaKkZ3EvcIpqGePF',
  'spotify:track:5ryWtuY9LJdqIKQcePENZ3',
  'spotify:track:025C5U3bZaCJmMsYYRpbbY',
  'spotify:track:1SrQkCfmF2ycMtAVpe4h6x',
  'spotify:track:4wZVrrMGdW7J2IPSDRRbRo',
  'spotify:track:6r5IvOcyN2NGNWGLC6AS5O',
  'spotify:track:5HDmqQZ3NYoiQkbrTris5y',
  'spotify:track:1EbklS2pjM17sxCPoofESx',
  'spotify:track:6lTQXJcbOYIg5fYivZou9R',
  'spotify:track:3cT1uCm6weuMywq8c14Awk',
  'spotify:track:6lGiuCuVtBEy9YgqfhhNIi',
  'spotify:track:5Qaxo1bNMJ73vLR058mZpt',
  'spotify:track:5kO3R1fxKjbWXpDU0TAdl4',
  'spotify:track:11natXGVNkPw4TdNG5M8GP',
  'spotify:track:1i1fxkWeaMmKEB4T7zqbzK',
  'spotify:track:0B4E0IrxBG77W80ZJQTy50',
  'spotify:track:1O5u9GJaKkZ3EvcIpqGePF',
  'spotify:track:5ryWtuY9LJdqIKQcePENZ3',
  'spotify:track:025C5U3bZaCJmMsYYRpbbY',
  'spotify:track:1SrQkCfmF2ycMtAVpe4h6x',
  'spotify:track:4wZVrrMGdW7J2IPSDRRbRo',
  'spotify:track:6r5IvOcyN2NGNWGLC6AS5O',
  'spotify:track:5HDmqQZ3NYoiQkbrT